In [1]:
import pickle,os
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from simpletransformers.classification import ClassificationModel
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import json
import numpy as np
import nltk

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
with open('models/vectorizer.pickle', 'rb') as vectorizerhandle:
    vectorizer = pickle.load(vectorizerhandle)
    vectorizerhandle.close()
with open('models/classes.pickle', 'rb') as classhandle:
    classes = pickle.load(classhandle)
    classhandle.close()


with open('models/NB_model.pickle', 'rb') as NB_modelhandle:
    NB_model = pickle.load(NB_modelhandle)
    NB_modelhandle.close()
with open('models/SVC_model.pickle', 'rb') as SVC_modelhandle:
    SVC_model = pickle.load(SVC_modelhandle)
    SVC_modelhandle.close()
with open('models/KNN_model.pickle', 'rb') as KNN_modelhandle:
    KNN_model = pickle.load(KNN_modelhandle)
    KNN_modelhandle.close()
with open('models/DecisionTree_model.pickle', 'rb') as DecisionTree_modelhandle:
    DecisionTree_model = pickle.load(DecisionTree_modelhandle)
    DecisionTree_modelhandle.close()
with open('models/RandomForest_model.pickle', 'rb') as RandomForest_modelhandle:
    RandomForest_model = pickle.load(RandomForest_modelhandle)
    RandomForest_modelhandle.close()
with open('models/LogisticRegression_model.pickle', 'rb') as LogisticRegression_modelhandle:
    LogisticRegression_model = pickle.load(LogisticRegression_modelhandle)
    LogisticRegression_modelhandle.close()


In [3]:
df = pd.read_csv('TextClassification_Data_without_nan.csv',usecols=['SUMMARY', 'categories'])

In [4]:
df.shape

(49663, 2)

In [5]:
df = df.dropna(axis=0, subset=['SUMMARY'])

In [6]:
df.shape

(49663, 2)

In [7]:
df.categories.value_counts()

PRESCRIPTION     14500
APPOINTMENTS     12960
ASK_A_DOCTOR     11743
MISCELLANEOUS    10460
Name: categories, dtype: int64

In [8]:
### shuffling the dataset
from sklearn.utils import shuffle
df = shuffle(df)
df = df.reset_index(drop=True)


In [9]:
df.head(50)

,SUMMARY,categories
0,Has devloped some jerking & tics while sleeping,ASK_A_DOCTOR
1,Phone Note,MISCELLANEOUS
2,new rx to FMS per pt request,ASK_A_DOCTOR
3,balance order,PRESCRIPTION
4,reschedule sleep due to snow missed,APPOINTMENTS
5,Please call to schedule the NCS/EMG.,MISCELLANEOUS
6,Appt scheduled,APPOINTMENTS
7,FROVA rxrf,ASK_A_DOCTOR
8,Patient needs ICD9 code included on disability...,MISCELLANEOUS
9,Caremark- Rx Tecfidera request,PRESCRIPTION


In [10]:
df.shape

(49663, 2)

In [11]:
messages = df.SUMMARY.tolist()

In [12]:
df.categories.value_counts()

PRESCRIPTION     14500
APPOINTMENTS     12960
ASK_A_DOCTOR     11743
MISCELLANEOUS    10460
Name: categories, dtype: int64

In [13]:
#Data cleaning and preprocessing
import re
import nltk
# nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
corpus = []

for i in range(0, len(messages)):
    message = re.sub('[^a-zA-Z]', ' ', str(messages[i]))
    message = message.lower()
    message = message.split()
    
    message = [lemmatizer.lemmatize(word) for word in message if not word in stopwords.words('english')]
    message = ' '.join(message)
    corpus.append(message)

In [14]:
################## preprocessed dafa for classical #######################
classical_transcription = vectorizer.transform(corpus).toarray()


################## classical classification #######################
'''
naive_bayes_prediction = NB_model.predict(classical_transcription)
svc_prediction = SVC_model.predict(classical_transcription)
knn_prediction = KNN_model.predict(classical_transcription)
decision_tree_prediction = DecisionTree_model.predict(classical_transcription)
random_forest_prediction = RandomForest_model.predict(classical_transcription)
logistic_regression_prediction = LogisticRegression_model.predict(classical_transcription)
'''




'\nnaive_bayes_prediction = NB_model.predict(classical_transcription)\nsvc_prediction = SVC_model.predict(classical_transcription)\nknn_prediction = KNN_model.predict(classical_transcription)\ndecision_tree_prediction = DecisionTree_model.predict(classical_transcription)\nrandom_forest_prediction = RandomForest_model.predict(classical_transcription)\nlogistic_regression_prediction = LogisticRegression_model.predict(classical_transcription)\n'

In [15]:
import time
t1 = time.time()

In [16]:
naive_bayes_prediction = NB_model.predict(classical_transcription)

In [17]:
t2 = time.time()
print('time require for naive_bayes prediction : ', (t2-t1)/3600)

time require for naive_bayes prediction :  0.0010485161675347222


In [18]:
svc_prediction = SVC_model.predict(classical_transcription)

In [19]:
t3 = time.time()
print('time require for svm prediction : ', (t3-t2)/3600)

time require for svm prediction :  2.386110435128212


In [20]:
knn_prediction = KNN_model.predict(classical_transcription)

In [21]:
t4 = time.time()
print('time require for knn prediction : ', (t4-t3)/3600)

time require for knn prediction :  3.6312304029199813


In [22]:
decision_tree_prediction = DecisionTree_model.predict(classical_transcription)

In [23]:
t5 = time.time()
print('time require for decision_tree prediction : ', (t5-t4)/3600)

time require for decision_tree prediction :  0.00032513731055789524


In [24]:
random_forest_prediction = RandomForest_model.predict(classical_transcription)

In [25]:
t6 = time.time()
print('time require for random_forest prediction : ', (t6-t5)/3600)

time require for random_forest prediction :  0.00208329094780816


In [26]:
logistic_regression_prediction = LogisticRegression_model.predict(classical_transcription)

In [27]:
t7 = time.time()
print('time require for logistic_regression prediction : ', (t7-t6)/3600)

time require for logistic_regression prediction :  0.00024347490734524197


In [28]:
naive_bayes_prediction.shape

(49663,)

In [29]:
bias = [1.0, 1.3, 0.7, 0.9, 1.1, 1.2]
#bias = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

ensenble_prediction = []
for i in range(len(naive_bayes_prediction)):
    predicted_class = [0, 0, 0, 0]
    predicted_class[naive_bayes_prediction[i]] = predicted_class[naive_bayes_prediction[i]] + bias[0]
    predicted_class[svc_prediction[i]] = predicted_class[svc_prediction[i]] + bias[1]
    predicted_class[knn_prediction[i]] = predicted_class[knn_prediction[i]] + bias[2]
    predicted_class[decision_tree_prediction[i]] = predicted_class[decision_tree_prediction[i]] + bias[3]
    predicted_class[random_forest_prediction[i]] = predicted_class[random_forest_prediction[i]] + bias[4]
    predicted_class[logistic_regression_prediction[i]] = predicted_class[logistic_regression_prediction[i]] + bias[5]
    


    max_index = predicted_class.index(max(predicted_class))
    #print(predicted_class)
    ensenble_prediction.append(classes[max_index])

In [30]:
wrong = 0
prescription = 0
appointments = 0
ask_a_doctor = 0
miscellaneous = 0
counter = 1
wrong_predicted=[]
for (i, j) in zip(df.categories, ensenble_prediction):
    if i != j:
        wrong = wrong + 1
        wrong_predicted.append(counter)
    if j == 'PRESCRIPTION':
        prescription = prescription + 1
    if j == 'APPOINTMENTS':
        appointments = appointments + 1
    if j == 'ASK_A_DOCTOR':
        ask_a_doctor = ask_a_doctor + 1
    if j == 'MISCELLANEOUS':
        miscellaneous = miscellaneous + 1
    
    #print(counter + 1)
    counter = counter + 1
    #print('{}  ------------- {}'.format(i, j))

print('wrong prediction {}'.format(wrong))
print('PRESCRIPTION prediction: {}'.format(prescription))
print('APPOINTMENTS: {}'.format(appointments))
print('ASK_A_DOCTOR prediction: {}'.format(ask_a_doctor))
print('MISCELLANEOUS prediction: {}'.format(miscellaneous))

wrong prediction 7499
PRESCRIPTION prediction: 13570
APPOINTMENTS: 12391
ASK_A_DOCTOR prediction: 11566
MISCELLANEOUS prediction: 12136


In [31]:
wrong_predicted

[6,
 13,
 14,
 17,
 21,
 22,
 32,
 33,
 43,
 44,
 45,
 52,
 67,
 68,
 73,
 78,
 91,
 93,
 98,
 99,
 111,
 133,
 139,
 141,
 143,
 146,
 159,
 167,
 172,
 177,
 193,
 194,
 195,
 213,
 217,
 218,
 221,
 225,
 230,
 232,
 234,
 238,
 285,
 288,
 290,
 298,
 302,
 307,
 309,
 312,
 318,
 320,
 324,
 332,
 334,
 336,
 350,
 353,
 370,
 371,
 373,
 389,
 391,
 394,
 395,
 399,
 409,
 412,
 413,
 423,
 428,
 438,
 446,
 447,
 448,
 450,
 456,
 457,
 461,
 468,
 472,
 475,
 487,
 489,
 493,
 494,
 496,
 498,
 507,
 515,
 517,
 525,
 535,
 540,
 541,
 548,
 558,
 559,
 577,
 596,
 597,
 599,
 600,
 606,
 613,
 616,
 639,
 651,
 652,
 653,
 657,
 663,
 669,
 670,
 679,
 687,
 691,
 710,
 713,
 714,
 721,
 723,
 729,
 735,
 742,
 745,
 748,
 752,
 753,
 757,
 764,
 781,
 784,
 785,
 796,
 799,
 807,
 812,
 824,
 826,
 827,
 837,
 839,
 860,
 862,
 867,
 891,
 898,
 902,
 903,
 904,
 909,
 921,
 928,
 950,
 952,
 965,
 971,
 972,
 973,
 980,
 982,
 986,
 990,
 994,
 997,
 1000,
 1001,
 1004,
 101

In [32]:
with open("TextClassification_Data_without_nan.csv", 'r', encoding='cp1252') as f1, open("TextClassification_Data_edited.csv", 'a') as f2 :
    line = f1.readline() #remove header
    f2.write(line)
    line = f1.readline()
    linenumber = 1
    while line != "" :
        if linenumber not in wrong_predicted:
            f2.write(line)
        line = f1.readline()
        linenumber = linenumber + 1

In [329]:
orthopedic_df.shape

(316, 2)

In [330]:
surgery_df.shape

(664, 2)

In [331]:
cardiovascular_df.shape

(318, 2)

In [333]:
consult_df.shape

(465, 2)

In [334]:
frames = [orthopedic_df, surgery_df, cardiovascular_df, consult_df]
df = pd.concat(frames)

In [335]:
df

,medical_specialty,transcription
0,Orthopedic,"TITLE OF OPERATION: , Youngswick osteotomy wit..."
1,Orthopedic,"PREOPERATIVE DIAGNOSES:,1. Hallux rigidus, le..."
2,Orthopedic,"CHIEF COMPLAINT:, Left wrist pain.,HISTORY OF..."
3,Orthopedic,"PREOPERATIVE DIAGNOSIS: , Wrist ganglion.,POST..."
4,Orthopedic,"PREOPERATIVE DIAGNOSIS:, T11 compression frac..."
...,...,...
460,Consult - History and Phy.,"HISTORY OF PRESENT ILLNESS: , The patient is a..."
461,Consult - History and Phy.,"REASON FOR REFERRAL:, The patient was referre..."
462,Consult - History and Phy.,"CHIEF COMPLAINT: , Chest pain.,HISTORY OF PRES..."
463,Consult - History and Phy.,"CHIEF COMPLAINT:, A 5-month-old boy with coug..."


In [336]:
df.to_csv('transcription_fresh.csv')


In [74]:
df.to_csv('TextClassification_Data_without_nan.csv')